In [2]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [31]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from termcolor import colored


print(colored('All libraries imported succesfully', 'green', attrs=["bold"]))

All libraries imported succesfully


In [22]:
warnings.filterwarnings('ignore')

print(colored('All libraries configed succesfully', 'green', attrs=['bold']))

All libraries configed succesfully


In [23]:
base_directory = '/home/nima/dataset-dw/dogs-vs-cats/'
train_folder = '/home/nima/dataset-dw/dogs-vs-cats/train/'
test_folder = '/home/nima/dataset-dw/dogs-vs-cats/test1/'

In [27]:
total_count = len(os.listdir(train_folder))
print('Number of samples in train folder : {}'.format(total_count))

Number of samples in train folder : 25000


In [37]:
cat_count, dog_count = 0, 0
for file_name in os.listdir(train_folder) :
    cat = re.findall("\Acat", file_name)
    dog = re.findall("\Adog", file_name)
    if cat :
        cat_count+=1
    if dog :
        dog_count+=1
print(colored(f'Number of cats : {cat_count}', 'blue', attrs=['bold']))
print(colored(f'Number of dogs : {dog_count}', 'blue', attrs=['bold']))

Number of cats : 12500
Number of dogs : 12500
